In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
train = pd.read_csv("../input/customer-analytics/Train.csv")

In [ ]:
train.head()

In [ ]:
train.groupby('Reached.on.Time_Y.N').mean()

In [ ]:
train.groupby('Reached.on.Time_Y.N').median()

In [ ]:
train.drop(train.columns[[0,10]], axis=1, inplace=True)

In [ ]:
train.Mode_of_Shipment[train.Mode_of_Shipment == 'Flight'] = 1
train.Mode_of_Shipment[train.Mode_of_Shipment == 'Ship'] = 2
train.Mode_of_Shipment[train.Mode_of_Shipment == 'Road'] = 3

In [ ]:
train.Product_importance[train.Product_importance == 'high'] = 1
train.Product_importance[train.Product_importance== 'low'] = 2
train.Product_importance[train.Product_importance == 'medium'] = 3

In [ ]:
train.Gender[train.Gender == 'F'] = 1
train.Gender[train.Gender== 'M'] = 2

In [ ]:
train.Warehouse_block[train.Warehouse_block == 'A'] = 1
train.Warehouse_block[train.Warehouse_block== 'B'] = 2
train.Warehouse_block[train.Warehouse_block == 'C'] = 3
train.Warehouse_block[train.Warehouse_block== 'D'] = 4
train.Warehouse_block[train.Warehouse_block== 'F'] = 5
train

In [ ]:
train["Mode_of_Shipment"]= train["Mode_of_Shipment"].astype(int) 
train["Product_importance"]= train["Product_importance"].astype(int) 
train["Gender"]= train["Gender"].astype(int) 
train["Warehouse_block"]= train["Warehouse_block"].astype(int) 

In [ ]:
train.info()

In [ ]:
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#from sklearn.preprocessing import MinMaxScaler
#X_norm = MinMaxScaler().fit_transform(X)
#X_new = SelectKBest(chi2, k=6).fit_transform(X, Y)
#X_new

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(train.drop('Reached.on.Time_Y.N',axis=1))
scaled_features = scaler.transform(train.drop('Reached.on.Time_Y.N',axis=1))

Xnew = pd.DataFrame(scaled_features,columns=train.columns[:-1])
Xnew.head()

In [ ]:
X=Xnew
Y=train['Reached.on.Time_Y.N']

In [ ]:
import statsmodels.api as sm
X=train.drop('Reached.on.Time_Y.N',axis=1)
Y=train['Reached.on.Time_Y.N']

#logit_model=sm.Logit(Y,X)
#result=logit_model.fit()
#print(result.summary2())

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20)

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train,y_train)
predictions=model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions))

## RFE Selection

In [ ]:
X=train.drop(columns=['Warehouse_block','Mode_of_Shipment','Reached.on.Time_Y.N'],axis=1)
Y=train['Reached.on.Time_Y.N']

In [ ]:
from sklearn.feature_selection import RFE
rfe = RFE(model, 6)
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,Y)  
#Fitting the data to model
model.fit(X_rfe,Y)
print(rfe.support_)

In [ ]:
X.head()

### Scaling out the data

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30)

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train,y_train)
predictions=model.predict(X_train)

In [ ]:
## To minimize the residual sum of squares between the observed targets in the dataset.

from sklearn.metrics import classification_report,confusion_matrix
print("Coefficients of the Logistic regression model")
coef = model.coef_
intercept = model.intercept_
print("Coef: ", coef)
print("Intercept: ", intercept)

In [ ]:
print("Confusion Matrix for LR model::")
conf_mat_lr = confusion_matrix(y_train.tolist(),predictions)
print(conf_mat_lr)
sns.heatmap(conf_mat_lr,annot = True,fmt="d",linewidths=.5) 
plt.xlabel('Predicted classes')
plt.ylabel('Actual classes')
plt.show()

# calculate accuracy scores for train sets
accuracy_lr = accuracy_score(y_train,predictions)
print("Accuracy score (train) for LR model::", accuracy_lr)

In [ ]:
## Test SET
predicted_test_classes_lr = model.predict(X_test)

## Confusion Matrix
print("Confusion Matrix (Test set) for LR model::")
conf_mat_test_lr = confusion_matrix(y_test.tolist(),predicted_test_classes_lr)
print(conf_mat_test_lr)

sns.heatmap(conf_mat_test_lr,annot = True,fmt="d",linewidths=.5) 
plt.xlabel('Predicted classes')
plt.ylabel('Actual classes')
plt.show()

accuracy_test_lr = accuracy_score(y_test,predicted_test_classes_lr)
print("Accuracy score - Log Reg (Test set)::", accuracy_test_lr)


In [ ]:
## ROC AUC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

plt.figure()
logit_roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1], drop_intermediate=False)
#, drop_intermediate=False
plt.plot(fpr, tpr, 'b--', label='Logistic Regression (area = %0.3f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1], color='orange', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()